# LLM-based Approaches for Forecasting: Reprogramming GPT-2

**Approximate Learning Time**:Up to 3 hours

---

In this notebook, we will learn about the recent work by [Jin et al. (2023)](https://arxiv.org/abs/2310.01728) on reprogramming an open-source LLM such as GPT-2 or LLaMA for time series forecasting. We will be using `neuralforecast` library to train TimeLLM model on our dataset.

--- 
## TimeLLM

We have already learned about LLMs in the previous notebook. Some LLMs are closed-source, meaning that their model weights and training data are not accessible, which limits their use to direct interaction. On the other hand, there are smaller LLMs, like GPT-2 or the LLaMA series, that are open-source, meaning both their model weights and architecture are accessible to everyone.

In their work, the authors leverage pre-trained models like GPT-2 or LLaMA by:

1. **Initializing the prompt**: The prompt is initialized with a basic description of the time series, including its characteristics and some statistics. This prompt serves as a prefix for the input.

2. **Processing the time series**: The time series is normalized and divided into patches using a sliding 1D window, where each window creates a patch of length $ L $. These patches are embedded into a dimension $ D $. The token embeddings of the LLM are also reduced to a smaller subspace using linear probing, transforming them from $ \mathcal{R}^{V \times D} $ to $ \mathcal{R}^{V' \times D} $, where $ V' \ll V $; the authors call these prototypes. Self-attention is then applied to map the query (time series patches) to the values (prototypes embeddings). The idea is to learn a relationship between the text and time series modalities, thereby enabling a cross-domain adaptations. These patch embeddings are then appended to the prompt embedding from the previous step.

3. **Generating predictions**: Once the LLM is prompted with the concatenated embeddings, the output embeddings are passed through a neural network tasked with predicting the next steps in the time series. 


The network then computes the loss and gradients are backpropagated. During backpropagation, gradients are computed throughout the model, including in the LLM. However, the LLM itself is kept frozen, meaning its weights are not updated. This approach allows the authors to train only a lightweight neural network on top of the LLM, leading to accurate time series forecasts.

Figure below is from the paper and depicts the above process visually: 

<div style="text-align: center; padding: 20px;">
<img src="images/timellm.png" style="max-width: 70%; clip-path: inset(2px); height: auto; border-radius: 15px; box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);">
</div>


Reading [paper](https://arxiv.org/abs/2310.01728) and going through the author's [GitHub repo](https://github.com/KimMeen/Time-LLM) are a good resource to understand further. 


**References**:

[(Jin et al. 2023) Time-LLM: Time Series Forecasting by Reprogramming Large Language Models](https://arxiv.org/abs/2310.01728)

---

Let's load the log daily returns of exchange rates, and split the data into train, validation, and test subsets!

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.models import TimeLLM
from transformers import GPT2Config, GPT2Model, GPT2Tokenizer

import pathlib
import pandas as pd


## WARNING: To compare different models on the same horizon, keep this same across the notebooks
from termcolor import colored
import sys; sys.path.append("../")
import utils

FORECASTING_HORIZON = [4, 8, 12, 24, 52] # weeks 
MAX_FORECASTING_HORIZON = max(FORECASTING_HORIZON)

SEQUENCE_LENGTH = 2 * MAX_FORECASTING_HORIZON
PREDICTION_LENGTH = MAX_FORECASTING_HORIZON

DIRECTORY_PATH_TO_SAVE_RESULTS = pathlib.Path('../results/DIY/').resolve()
MODEL_NAME = "TimeLLM"

RESULTS_DIRECTORY = DIRECTORY_PATH_TO_SAVE_RESULTS / MODEL_NAME
if RESULTS_DIRECTORY.exists():
    print(colored(f'Directory {str(RESULTS_DIRECTORY)} already exists.'
           '\nThis notebook will overwrite results in the same directory.'
           '\nYou can also create a new directory if you want to keep this directory untouched.'
           ' Just change the `MODEL_NAME` in this notebook.\n', "red" ))
else:
    RESULTS_DIRECTORY.mkdir(parents=True)

data, transformed_data = utils.load_tutotrial_data(dataset='exchange_rate', log_transform=True)
data = transformed_data 

train_val_data = data.iloc[:-MAX_FORECASTING_HORIZON]
train_data, val_data = train_val_data.iloc[:-MAX_FORECASTING_HORIZON], train_val_data.iloc[-MAX_FORECASTING_HORIZON:]
test_data = data.iloc[-MAX_FORECASTING_HORIZON:]
print(f"Number of steps in training data: {len(train_data)}\nNumber of steps in validation data: {len(val_data)}\nNumber of steps in test data: {len(test_data)}")

%load_ext autoreload
%autoreload 2

----

## Data Handling for Neuralforecast

The data format requirements can be found [here](https://nixtlaverse.nixtla.io/neuralforecast/docs/tutorials/getting_started_complete.html). The format requires three specific keys:
- `unique_id`
- `ds`
- `y`

Therefore, we will convert our dataframe into the format required by `neuralforecast` to proceed with the training.


In [2]:
train_val_data

nf_train_val_data = pd.melt(train_val_data.reset_index(), id_vars=['date'], value_vars=train_val_data.columns, value_name='y')

nf_train_val_data = nf_train_val_data.rename(columns={
        'date': 'ds',
        'variable': 'unique_id'
    }
)


--- 

## TimeLLM via Neuralforeacast 

In **this tutorial**, we will use TimeLLM's implementation from the `neuralforecast` library, which provides an interface similar to Lightning framework. While it is not the most polished option, as the library is still under development, it offers a convenient starting point. As of September 2024, there aren't many other libraries available for TimeLLM, so it’s worthwhile to check if a more updated version of `neuralforecast` is available or explore alternative libraries that might offer better support for TimeLLM.

Implementation of the model in `neuralforecast` can be found [here](https://github.com/Nixtla/neuralforecast/blob/main/nbs/models.timellm.ipynb).


The model loading might take around 10-15 minutes depending on the processor speed. 


In [ ]:
# define the open-sourced LLM that we will use 
gpt2_config = GPT2Config.from_pretrained('openai-community/gpt2')
gpt2 = GPT2Model.from_pretrained('openai-community/gpt2', config=gpt2_config)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('openai-community/gpt2')

# define the model
prompt_prefix = "The dataset contains data on exchange rate across 8 countries."

timellm = TimeLLM(
            h=MAX_FORECASTING_HORIZON,
            input_size=SEQUENCE_LENGTH,
            llm=gpt2,
            llm_config=gpt2_config,
            llm_tokenizer=gpt2_tokenizer,
            prompt_prefix=prompt_prefix,
            batch_size=64,
            valid_batch_size=4,
            max_steps=1000, # will determine the time to train 
        )

# let nf know what are the models
nf = NeuralForecast(
    models=[timellm],
    freq=data.index.freq,
)

# fit it on data
nf.fit(df=nf_train_val_data, val_size=MAX_FORECASTING_HORIZON, verbose=True)

--- 

## Forecast 

In [ ]:
forecasts = nf.predict(df=nf_train_val_data)

# 
df = forecasts.reset_index()
df = df.pivot(columns=['unique_id'], index='ds')
df.columns = df.columns.droplevel(0)
df.index.name = 'date'

test_predictions_df = df
AUGMENTED_COL_NAMES = [f"{MODEL_NAME}_{col}_mean" for col in data.columns]
test_predictions_df.columns = AUGMENTED_COL_NAMES

# ssave them to the directory
test_predictions_df.to_csv(f"{str(RESULTS_DIRECTORY)}/predictions.csv", index=True)
print(test_predictions_df.shape)
test_predictions_df.head()

--- 

## Evaluate 

Let's compute the metrics by comparing the predictions with that of the target data. Note that we will have to rename the columns of the dataframe to match the expected column names by the function. 

In [ ]:
# evalaute metrics
target_data = data[-MAX_FORECASTING_HORIZON:]
model_metrics, records = utils.get_mase_metrics(
    historical_data=train_val_data,
    test_predictions=test_predictions_df.rename(
            columns={x:x.split("_")[1] for x in test_predictions_df.columns
        }),
    target_data=target_data,
    forecasting_horizons=FORECASTING_HORIZON,
    columns=data.columns, 
    model_name=MODEL_NAME
)
records = pd.DataFrame(records)

records.to_csv(f"{str(RESULTS_DIRECTORY)}/metrics.csv", index=False)
records[['col', 'horizon', 'mase']].pivot(index=['horizon'], columns='col')

--- 

## Compare Models

In [ ]:
utils.display_results(path=DIRECTORY_PATH_TO_SAVE_RESULTS, metric='mase')

--- 
## Plot Forecasts

In [ ]:
fig, axs = utils.plot_forecasts(
    historical_data=train_val_data,
    forecast_directory_path=DIRECTORY_PATH_TO_SAVE_RESULTS,
    target_data=target_data,
    columns=data.columns,
    n_history_to_plot=10, 
    forecasting_horizon=MAX_FORECASTING_HORIZON,
    dpi=200
)

--- 

## Conclusion

We learned about TimeLLM, an approach to reprogram open-sourced LLMs for time series forecasting.

---

## Exercises

- Use LLaMA instead of GPT-2 for the model.

- Select and optimize hyperparameters for the model, such as the embedding dimension and others, to achieve the best performance.

- Apply a normalization procedure (e.g., **min-max scaling**) to the data, ensuring that only the training data is used for fitting the scaler. Perform the modeling process on the normalized data and, after generating the final model's predictions, invert the normalization to return the output to its original scale. See `sklearn.preprocessing.MinMaxScaler` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html))

- Additionally, perform the modeling on the **raw data**, without applying any transformation (such as converting it into log daily returns), to compare results directly with the untransformed dataset.


---

## Next Steps

Kudos! You've made it to the end of the tutorial. I hope you're now feeling more confident navigating the complex landscape of forecasting methods. Best of luck in your future forecasting endeavors!

If you have a moment, I'd really appreciate your feedback. You can share your thoughts using the survey link provided in `0_introduction.ipynb`. Thank you!

---